In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

from code.prepare_data import X, y
from code.utils import root_mean_square_percentage_error, compare_train_test_error

pd.options.display.max_columns = None

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=False)
# Ensure we are not using future to predict past
assert X_train["Year"].max() <= X_test["Year"].min()

model = RandomForestRegressor(
    n_estimators=10, max_depth=1000, random_state=42, n_jobs=-1)
model.fit(X, y)

X_train["SalesPrediction"] = model.predict(X_train)

/Users/jack/code/dsr/dsr_env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [24]:
assert X_train.shape[0] == y_train.shape[0]
X_train["Sales"] = y_train
X_train["Error"] = np.abs(X_train["SalesPrediction"] - y_train)

/Users/jack/code/dsr/dsr_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jack/code/dsr/dsr_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
X_train.head()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,DayOfWeek,Customers,Open,Promo,SchoolHoliday,HasCompetition,Week,Month,Year,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,PromoInterval_-1,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,SalesPrediction,Sales,Error
59047,76,19960.0,3,2006,0,-1,-1,5,982,1,1,0,1,20,5,2013,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,9190.7,9306,115.3
122633,159,8530.0,3,2013,0,-1,-1,5,802,1,1,0,1,20,5,2013,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,9425.0,8997,428.0
48131,62,2050.0,-1,-1,0,-1,-1,5,773,1,1,0,0,20,5,2013,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,7061.6,7117,55.4
10293,13,310.0,-1,-1,1,45,2009,5,454,1,1,0,0,20,5,2013,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,6315.7,6336,20.3
122011,158,11840.0,-1,-1,1,31,2009,5,562,1,1,0,0,20,5,2013,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,8424.8,8602,177.2


In [32]:
X_train.sort_values(by="Error",ascending=False).head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 605238 entries, 59047 to 776405
Data columns (total 34 columns):
Store                             605238 non-null int64
CompetitionDistance               605238 non-null float64
CompetitionOpenSinceMonth         605238 non-null int64
CompetitionOpenSinceYear          605238 non-null int64
Promo2                            605238 non-null int64
Promo2SinceWeek                   605238 non-null int64
Promo2SinceYear                   605238 non-null int64
DayOfWeek                         605238 non-null int64
Customers                         605238 non-null int64
Open                              605238 non-null int64
Promo                             605238 non-null int64
SchoolHoliday                     605238 non-null int64
HasCompetition                    605238 non-null int64
Week                              605238 non-null int64
Month                             605238 non-null int64
Year                              605238 no

In [37]:
import datetime

def competition_length(row):
    year = int(row.CompetitionOpenSinceYear)
    month = int(row.CompetitionOpenSinceMonth)
    if year == -1:
        return 0
    else:
        competition_start = datetime.datetime(year=year, month=month, day=1)
        duration = row.Date - competition_start
        return duration.days

X_train["CompetitionLengthInDays"] = X_train.apply(competition_length, axis=1)
X_train.head()

/Users/jack/code/dsr/dsr_env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,DayOfWeek,Customers,Open,Promo,SchoolHoliday,HasCompetition,Week,Month,Year,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,PromoInterval_-1,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,SalesPrediction,Sales,Error,CompetitionLength
59047,76,19960.0,3,2006,0,-1,-1,5,982,1,1,0,1,20,5,2013,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,9190.7,9306,115.3,2006-03-01 00:00:00
122633,159,8530.0,3,2013,0,-1,-1,5,802,1,1,0,1,20,5,2013,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,9425.0,8997,428.0,2013-03-01 00:00:00
48131,62,2050.0,-1,-1,0,-1,-1,5,773,1,1,0,0,20,5,2013,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,7061.6,7117,55.4,0
10293,13,310.0,-1,-1,1,45,2009,5,454,1,1,0,0,20,5,2013,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,6315.7,6336,20.3,0
122011,158,11840.0,-1,-1,1,31,2009,5,562,1,1,0,0,20,5,2013,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,8424.8,8602,177.2,0


In [ ]:
def promo_length_weeks(row):
    year = int(row.Promo2SinceYear)
    month = int(row.Promo2SinceWeek)
    if year == -1:
        return 0
    else:
        return datetime.datetime(year=year, month=month, day=1)